In [1]:
include("EMREfunctionsCompleteSS.jl");

  Activating project at `~/Documents/EMRE Julia Package`


In [189]:
# function to_timedependent_symbol(symbol::Symbol, vec::Vector)
#     @variables t
#     return 
#         ModelingToolkit.variable(symbol, join(vec); T=ModelingToolkit.FnType{Tuple{Any},Real})
#      (
#         t
#     )
# end

rn = @reaction_network begin
    @parameters k
    @species A(t) B(t)

    k, 0 --> A
end

# # display(collect(keys(rn.var_to_name)))

# # a = to_timedependent_symbol(collect(keys(rn.var_to_name))[1], [1])

# a = to_timedependent_symbol(collect(keys(rn.var_to_name))[findfirst(x->isequal(x, rn.species[2]), collect(values(rn.var_to_name)))], [1]);

# @species $a
# # Reaction(rn.ps[1], [a], [a], [1], [1])


rn.species[1]



A(t)

In [268]:
# Define a reaction network with species A(t)
rn = @reaction_network begin
    @parameters k
    @species A(t)

    k, 0 --> A
end

# Define a new species with the same letter but a subscript
function create_subscripted_species(symbol::Symbol, subscript::Int)
    # Create a new symbol with the subscript using Unicode subscript characters
    subscripted_symbol =  Symbol(string(string(symbol), string(subscript)))  # "₁" is the subscripted 1 in Unicode
    return subscripted_symbol
end

# Create a new species A₁(t) based on species A(t)
# test = create_subscripted_species(:A, 1)

# test2 = Catalyst.Species(test)

# Add the new species to the reaction network (or do anything else with it)
# @species $A₁(t)

# You can now use `A₁` as a new species in the reaction network.

create_subscripted_species (generic function with 1 method)

In [274]:
rn = @reaction_network begin
    @parameters k
    @species A(t) B(t)

    k, 0 --> A
end

numVoxels = 2;
newSpecies = []
for species in rn.species
    varIndex = findfirst(x->isequal(x, species), collect(values(rn.var_to_name)));
    speciesSym = collect(keys(rn.var_to_name))[varIndex]
    for i in 1:numVoxels
        subscriptedSpecies = create_subscripted_species(speciesSym, i)
        @species $subscriptedSpecies(t)
        push!(newSpecies, subscriptedSpecies)
    end
end
# typeof(newSpecies[1])

# @species $newSpecies[1]


# Reaction(k, [newSpecies[1]], [newSpecies[2]], [1], [1])

In [304]:
# asymbol = :A
# @variables t
# Asymbolic = (@species ($asymbol)(t))[1]

# Catalyst.isspecies(Asymbolic)

# str = "ABC"
# temp = Symbol(str)
# Temp = (@species ($temp)(t))[1]

# Catalyst.isspecies(Temp)

# Define a new species with the same letter but a subscript
function create_subscripted_species(symbol::Symbol, subscript::Int)
    # Create a new symbol with the subscript using Unicode subscript characters
    subscripted_symbol =  string(string(symbol), string(subscript))  # "₁" is the subscripted 1 in Unicode
    return subscripted_symbol
end;

rn = @reaction_network begin
    @parameters k
    @species A(t) B(t)

    k, 0 --> A
end

numVoxels = 2;
newSpecies = []
for species in rn.species
    varIndex = findfirst(x->isequal(x, species), collect(values(rn.var_to_name)));
    speciesSym = collect(keys(rn.var_to_name))[varIndex]
    for i in 1:numVoxels
        subscriptedSpecies = create_subscripted_species(speciesSym, i)
        symbVersion = Symbol(subscriptedSpecies)
        push!(newSpecies, (@species ($symbVersion)(t))[1])
    end
end

display(newSpecies)
display(Catalyst.isspecies(newSpecies[1]))

Reaction(rn.ps[1], [newSpecies[1]], [newSpecies[2]], [1], [1])

4-element Vector{Any}:
 A1(t)
 A2(t)
 B1(t)
 B2(t)

true

k, A1 --> A2

In [1]:
include("EMREfunctionsCompleteSS.jl");

  Activating project at `~/Documents/EMRE Julia Package`


In [41]:
function createSubSpecies(symbol::Symbol, voxelNum::Int)
    subSymbol =  string(string(symbol)[1], string(voxelNum))
    return Symbol(subSymbol)
end;



In [2]:
struct VoxelReaction
    rxn::Reaction
    voxelNums::Vector{Int}
end

In [14]:
struct VoxelSpecies
    speciesName::Num
    voxelsNums::Vector{Int}
end;


In [104]:
struct DiffReaction
    speciesName::Num
    voxelPairs::Vector{Vector{Int}}
end;

In [113]:
@species A B
@parameters k d
@variables t

a = VoxelSpecies(A, [1,2,3])
b = VoxelSpecies(B, [1,3])

aDiff = DiffReaction(A, [[1,2],[2,3]])

vr = VoxelReaction((@reaction k, A-->B), [1,3]);

In [123]:
rn = @reaction_network begin
end;


## for each type of reaction: 
for voxelNum in vr.voxelNums ## for each voxel the reaction happens in
    voxelSubstrates = []; ## make subspecies of the reactants
    for sub in vr.rxn.substrates
        subSpeciesSymb = createSubSpecies(Symbol(sub), voxelNum)
        subSpecies = (@species ($subSpeciesSymb)(t))[1]
        push!(voxelSubstrates, subSpecies)
        addspecies!(rn, subSpecies);
    end;
    voxelProducts = []; ## make subspecies of the products
    for prod in vr.rxn.products
        subSpeciesSymb = createSubSpecies(Symbol(prod), voxelNum)
        subSpecies = (@species ($subSpeciesSymb)(t))[1]
        push!(voxelProducts, subSpecies)
        addspecies!(rn, subSpecies);
    end;
    react = Reaction(vr.rxn.rate, voxelSubstrates, voxelProducts, vr.rxn.substoich, vr.rxn.prodstoich)
    addreaction!(rn, react)
end;
addparam!(rn, vr.rxn.rate);
## end;

## for each species:
for voxelNum in a.voxelsNums
    subSpeciesSymb = createSubSpecies(Symbol(a.speciesName), voxelNum)
    subSpecies = (@species ($subSpeciesSymb)(t))[1]

    addspecies!(rn, subSpecies)
    # added = findfirst(x->isequal(x, subSpecies), rn.species)
    # if added == nothing
    #     addspecies!(rn, subSpecies)
    # end;
end;


## for each diffusion reaction:
for pair in aDiff.voxelPairs
    subSpeciesSymb1 = createSubSpecies(Symbol(aDiff.speciesName), pair[1])
    subSpecies1 = (@species ($subSpeciesSymb1)(t))[1]
    subSpeciesSymb2 = createSubSpecies(Symbol(aDiff.speciesName), pair[2])
    subSpecies2 = (@species ($subSpeciesSymb2)(t))[1]

    addspecies!(rn, subSpecies1)
    addspecies!(rn, subSpecies2)

    react1 = Reaction(d, [subSpecies1], [subSpecies2]);
    react2 = Reaction(d, [subSpecies2], [subSpecies1]);
    addreaction!(rn, react1)
    addreaction!(rn, react2)
end;

## end;

display(convert(ODESystem, rn, combinatoric_ratelaws=false))


Model ##ReactionSystem#868 with 5 equations
States (5):
  A1(t)
  B1(t)
  A3(t)
  B3(t)
⋮
Parameters (1):
  k